In [ ]:
import requests
from bs4 import BeautifulSoup
import string
import os

page = int(input())
type_article = input()
parents_link = os.getcwd()
url = 'https://www.nature.com/nature/articles?searchType=journalSearch&sort=PubDate&page=1'
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')
articles = soup.find_all('li', class_='app-article-list-row__item')
articles_list = []
for N in range(page):
    new_dir = 'page_'+str(N+1)
    os.mkdir(new_dir)
    os.chdir(new_dir)
    for article in articles:
        if article.find('span', class_='c-meta__type', string=type_article):
            #find name new link
            title = article.find('h3', class_='c-card__title').text.strip()
            for element in string.punctuation:
                if element in title:
                    title = title.replace(element, '')
            title = title.replace(' ', '_')
            name_website = title + '.txt'
            articles_list.append(name_website)
            #parser information from important article
            new_url = 'https://www.nature.com' + article.find('a')['href']
            req_article = requests.get(new_url)
            new_soup = BeautifulSoup(req_article.content, 'html.parser')
            body = new_soup.find('div', class_='article-item__body')
            if not body:
                body = new_soup.find('div', class_='c-article-body u-clearfix')
            body = body.text.strip()
            #create file and add to file information
            new_file = open(name_website, 'w', encoding='utf-8')
            new_file.write(body)
            new_file.close()
    #print new list with new file name
    print(articles_list)
    articles_list = []
    #create parser for next page in website
    buttons = soup.find_all('a', class_='c-pagination__link')
    url = 'https://www.nature.com' + buttons[len(buttons)-1]['href']
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    articles = soup.find_all('li', class_='app-article-list-row__item')
    #return to parents direction
    os.chdir(parents_link)